In [3]:
import pandas as pd
import numpy as np

BASE_DIR='/home/thanuja/Dropbox/Lansing/'
#geo_file = 'lansing_geocodes.csv'
geo_file = 'fec_geocodes.csv'


census_blocks = pd.read_csv(BASE_DIR + geo_file)
census_blocks = census_blocks[census_blocks['Match Indicator'] == 'Match']

census_blocks

,ID,Input Address,Match Indicator,Match Type,Output Address,Longitude,Latitude,County Code,Tract Code,Block Code
0,22912,"221 Primrose Ln, Mason, MI,",Match,Exact,"221 PRIMROSE LN, MASON, MI, 48854",-84.512880,42.576515,65.0,6401.0,2008.0
3,20097,"7444 Delta Commerce Drive, Lansing, MI,",Match,Exact,"7444 DELTA COMMERCE DR, LANSING, MI, 48917",-84.659940,42.745975,45.0,20306.0,1000.0
4,23721,"1122 E Thomas L PKWY, Lansing, MI,",Match,Exact,"1122 E THOMAS L PKWY, LANSING, MI, 48917",-84.605890,42.746613,45.0,20103.0,2017.0
5,61769,"139 Woodmere Ave APT 16, East Lansing, MI,",Match,Exact,"139 WOODMERE AVE, EAST LANSING, MI, 48823",-84.468200,42.730470,65.0,4302.0,1000.0
6,57063,"912 Sybil St Unit 3, Ann Arbor, MI,",Match,Exact,"912 SYBIL ST, ANN ARBOR, MI, 48104",-83.743100,42.270680,161.0,400500.0,3003.0
...,...,...,...,...,...,...,...,...,...,...
5045,22501,"3830 S Waverly Road, Lansing, MI,",Match,Non_Exact,"3830 S WAVERLY RD, LANSING, MI, 48911",-84.603120,42.693287,45.0,20202.0,4007.0
5046,21733,"4220 Lochinver Circle, Lansing, MI,",Match,Exact,"4220 LOCHINVER CIR, LANSING, MI, 48911",-84.614090,42.690872,45.0,20202.0,2006.0
5047,21777,"3200 S. Washington Avenue Apt. 321, Lansing, MI,",Match,Non_Exact,"3200 S WASHINGTON AVE, LANSING, MI, 48910",-84.562126,42.700317,65.0,3700.0,1003.0
5048,22142,"1523 Georgetown, Lansing, MI,",Match,Non_Exact,"1523 GEORGETOWN BLVD, LANSING, MI, 48911",-84.573970,42.664070,65.0,5201.0,1000.0


In [28]:
fec_tract_df = census_blocks.groupby('Tract Code', as_index=False).agg(
    num_addrs=pd.NamedAgg(column='Tract Code', aggfunc=len),
    lat_avg=pd.NamedAgg(column='Latitude', aggfunc=np.mean),
    long_avg=pd.NamedAgg(column='Longitude', aggfunc=np.mean)
).sort_values('num_addrs', ascending=False)
    
fec_block_df = census_blocks.groupby(['Tract Code', 'Block Code'], as_index=False).agg(
    num_addrs=pd.NamedAgg(column='Block Code', aggfunc=len),
    lat_avg=pd.NamedAgg(column='Latitude', aggfunc=np.mean),
    long_avg=pd.NamedAgg(column='Longitude', aggfunc=np.mean)
).sort_values('num_addrs', ascending=False)

fec_tract_block_counts_df = fec_block_df.groupby('Tract Code', as_index=False).agg(
    num_blocks=pd.NamedAgg(column='Block Code', aggfunc=len))\
.sort_values('num_blocks', ascending=False)
fec_tract_df = fec_tract_block_counts_df.merge(fec_tract_df, on='Tract Code')


# At least one address per block in tract
#fec_filter_df = fec_tract_df[fec_tract_df['num_addrs'] >= 30].sort_values('Tract Code')
#tract_df[tract_df['ID'] > 10]
fec_tract_block_counts_df
#fec_block_df[fec_block_df['num_addrs'] >= 10]
#filter_df
#census_blocks[['Tract Code', 'Block Code', 'ID']]
fec_tract_df

,Tract Code,num_blocks,num_addrs,lat_avg,long_avg
0,7000.0,57,106,42.712158,-84.567403
1,2000.0,45,153,42.715241,-84.548697
2,6800.0,43,118,42.747859,-84.567313
3,800.0,37,86,42.751169,-84.538661
4,3700.0,33,255,42.690206,-84.569630
...,...,...,...,...,...
173,300.0,1,1,43.013645,-85.627570
174,20302.0,1,1,42.752434,-84.681460
175,31501.0,1,1,42.821854,-84.218630
176,10400.0,1,1,42.813885,-84.763626


In [22]:
violation_addresses = pd.read_csv(BASE_DIR + 'lansing_geocodes.csv')
violation_addresses = violation_addresses[violation_addresses['Match Indicator'] == 'Match']

violation_block_df = violation_addresses.groupby(['Tract Code', 'Block Code'], as_index=False).agg(
    num_addrs=pd.NamedAgg(column='Block Code', aggfunc=len),
    lat_avg=pd.NamedAgg(column='Latitude', aggfunc=np.mean),
    long_avg=pd.NamedAgg(column='Longitude', aggfunc=np.mean)
).sort_values('num_addrs', ascending=False)

violation_addresses = violation_addresses.merge(fec_block_df, on=['Tract Code', 'Block Code'], how='left')
# 15157 rows before merge
violation_addresses['num_addrs'].fillna(0, inplace=True)
#violation_addresses.sort_values('num_fec_addrs', ascending=False)
violation_addresses[violation_addresses['num_addrs'] >= 5]

violation_block_df
#violation_addresses

,Tract Code,Block Code,num_addrs,lat_avg,long_avg
406,2101.0,1008.0,137,42.720916,-84.529709
261,1200.0,1008.0,71,42.726844,-84.516959
984,3700.0,4000.0,63,42.695314,-84.574869
978,3700.0,3003.0,62,42.685402,-84.573681
279,1200.0,2007.0,61,42.727020,-84.526047
...,...,...,...,...,...
815,3301.0,2005.0,1,42.764084,-84.599174
812,3301.0,2002.0,1,42.768795,-84.600920
1293,6700.0,3006.0,1,42.732227,-84.570520
1296,6700.0,3009.0,1,42.732033,-84.571594


In [14]:
violation_fec_merged_df = pd.read_csv(BASE_DIR + 'City_of_Lansing/merged_from_violation_and_FEC.csv')
columns = ['enforced_address', 'owner_yn', 'age', 'gender', 'race', 'ethnicity', 'latitude', 'longitude']
violation_fec_merged_df = violation_fec_merged_df[columns + ['ID']].groupby(columns, as_index=False)\
    .count().rename(columns={'ID': 'num_violations'})
violation_fec_merged_df

,enforced_address,owner_yn,age,gender,race,ethnicity,latitude,longitude,num_violations
0,1001 BELAIRE AVE,0,33.0,Female,African-American/ Black,Not Hispanic/Latinx,42.684163,-84.566368,9
1,1001 MAHLON ST,0,42.0,Female,African-American/ Black,Not Hispanic/Latinx,42.744592,-84.531134,2
2,1001 MCCULLOUGH ST,1,64.0,Male,White,Not Hispanic/Latinx,42.723922,-84.529408,3
3,1002 BEECH ST,1,65.0,Female,White,Not Hispanic/Latinx,42.736700,-84.523400,1
4,1003 S GRAND AVE,1,39.0,Female,African-American/ Black,Not Hispanic/Latinx,42.693856,-84.566429,1
...,...,...,...,...,...,...,...,...,...
762,936 W MILLER RD,1,42.0,Male,African-American/ Black,Not Hispanic/Latinx,42.669505,-84.566180,5
763,940 MCCULLOUGH ST,0,58.0,Female,African-American/ Black,Not Hispanic/Latinx,42.724189,-84.530054,9
764,940 MCCULLOUGH ST,1,58.0,Female,African-American/ Black,Not Hispanic/Latinx,42.724189,-84.530054,1
765,946 VINCENT CT,0,25.0,Male,African-American/ Black,Not Hispanic/Latinx,42.687377,-84.566491,1


In [15]:
st_dict = {' AVENUE':' AVE', ' BOULEVARD':' BLVD', ' CENTER':' CTR', 
          ' CIRCLE':' CIR', ' COURT':' CT', ' DRIVE':' DR', ' EXPRESSWAY':' EXPY', 
          ' HEIGHTS':' HTS', ' HIGHWAY':' HWY', ' ISLAND':' IS', ' JUNCTION':' JCT', ' LAKE':' LK', 
          ' LANE':' LN', ' MOUNTAIN':' MTN', ' PARKWAY':' PKWY', ' PLACE':' PL', ' PLAZA':' PLZ',
          ' RIDGE':' RDG', ' ROAD':' RD', ' SQUARE':' SQ', ' STREET':' ST', ' STATION':' STA', 
          ' TERRACE':' TER', ' TRAIL':' TRL', ' TURNPIKE':' TPKE', ' VALLEY':' VLY', 
          ' APARTMENT':' APT', ' ROOM':' RM', ' SUITE':' STE', 
          ' NORTH':' N', ' EAST':' E', ' SOUTH':' S', ' WEST':' W', 
          ' NORTHEAST':' NE', ' NORTHWEST':' NW', ' SOUTHEAST':' SE', ' SOUTHWEST':' SW'} 

census_blocks['Input Address'] = census_blocks['Input Address'].str.upper()

for k, v in st_dict.items():
    census_blocks['Input Address'] = census_blocks['Input Address'].str.replace(k, v, regex=False)

In [16]:
import numpy as np

merged_lat_long_np = violation_fec_merged_df[['latitude', 'longitude']].to_numpy()
print(merged_lat_long_np)
fec_lat_long_np = census_blocks[['Latitude', 'Longitude']].to_numpy()
print(fec_lat_long_np)
match_count = 0
mismatch_count = 0
max_valid_distance = 0
for idx, lat_long in enumerate(merged_lat_long_np):
    distance = (fec_lat_long_np[:, 0] - lat_long[0]) ** 2 + (fec_lat_long_np[:, 1] - lat_long[1]) ** 2
    best = np.argmin(distance)
    violation_address = violation_fec_merged_df.iloc[idx]['enforced_address']
    fec_address = census_blocks.iloc[best]['Input Address'].split(',')[0]
    if not violation_address in fec_address and fec_address not in violation_address:
        print('x', best, distance[best], violation_address, fec_address)
        mismatch_count += 1
    else:
        match_count += 1
        max_valid_distance = max(max_valid_distance, distance[best])
print('match count', match_count, 'mismatch count', mismatch_count, 'max valid', max_valid_distance)

[[ 42.684163 -84.566368]
 [ 42.744592 -84.531134]
 [ 42.723922 -84.529408]
 ...
 [ 42.724189 -84.530054]
 [ 42.687377 -84.566491]
 [ 42.723912 -84.525316]]
[[ 42.576515 -84.51288 ]
 [ 42.745975 -84.65994 ]
 [ 42.746613 -84.60589 ]
 ...
 [ 42.700317 -84.562126]
 [ 42.66407  -84.57397 ]
 [ 42.699844 -84.56318 ]]
x 636 5.382800000189579e-08 1001 BELAIRE AVE 1009 BELAIRE AVE
x 590 4.765000000083305e-07 1002 BEECH ST 250 HORTON ST
x 2803 8.104500000327478e-08 1003 S GRAND AVE 1005 LEGRAND
x 2156 9.202099999866933e-08 1007 PORTER ST 1013 PORTER ST
x 990 3.130099999942984e-08 1008 QUEEN ST 1012 QUEEN ST.
x 351 4.6092999998420695e-08 1009 CADY CT 1021 CADY CT
x 590 4.765000000083305e-07 1010 BEECH ST 250 HORTON ST
x 1464 1.2456099999899087e-07 1010 E NORTH ST 1010 E N ST
x 590 4.765000000083305e-07 1015 HICKORY ST 250 HORTON ST
x 590 4.765000000083305e-07 1015 HICKORY ST 250 HORTON ST
x 937 5.637320000004864e-07 1016 HICKORY ST 1016 BEMENT ST
x 2156 1.1184500000151118e-07 1018 N PENNSYLVANIA A

In [33]:
violation_plot = violation_block_df.copy()
violation_plot['category'] = 'Violation'
#print(violation_plot)
total_violation_addrs = violation_plot['num_addrs'].sum()

fec_plot = fec_block_df.copy()
fec_plot['category'] = 'FEC'
#print(fec_plot)
total_fec_addrs = fec_plot['num_addrs'].sum()
print(total_fec_addrs)
# Make sum of two dataframes similar so that violations doesn't visually overwhelm the FEC data.
correction_factor = total_violation_addrs / total_fec_addrs
fec_plot['num_addrs'] *= correction_factor

print(total_violation_addrs, total_fec_addrs, correction_factor, fec_plot['num_addrs'].sum())

plot_df = pd.concat([fec_plot, violation_plot])
plot_df

3623
15157 3623 4.183549544576318 15156.999999999996


,Tract Code,Block Code,num_addrs,lat_avg,long_avg,category
1021,6600.0,2039.0,263.563621,42.737665,-84.544610,FEC
1291,20202.0,4007.0,251.012973,42.693287,-84.603120,FEC
586,3700.0,1008.0,217.544576,42.694146,-84.564579,FEC
221,2000.0,3004.0,188.259730,42.714302,-84.551967,FEC
594,3700.0,2003.0,154.791333,42.685154,-84.565412,FEC
...,...,...,...,...,...,...
815,3301.0,2005.0,1.000000,42.764084,-84.599174,Violation
812,3301.0,2002.0,1.000000,42.768795,-84.600920,Violation
1293,6700.0,3006.0,1.000000,42.732227,-84.570520,Violation
1296,6700.0,3009.0,1.000000,42.732033,-84.571594,Violation


In [34]:
import plotly.express as px
fig = px.scatter_mapbox(violation_plot, 
                        lat='lat_avg', 
                        lon='long_avg', 
                        #color='category',
                        #color_discrete_sequence=px.colors.qualitative.Pastel2,
                        color_discrete_sequence=['green'],
                        size = 'num_addrs',
                        zoom=11.3, 
                        height=800,
                        width=800,
                        title = "")

fig.update_layout(mapbox_style="open-street-map")
fig.show()

fig = px.scatter_mapbox(fec_plot, 
                        lat='lat_avg', 
                        lon='long_avg', 
                        #color='category',
                        #color_discrete_sequence=px.colors.qualitative.Pastel2,
                        color_discrete_sequence=['purple'],
                        size = 'num_addrs',
                        zoom=11.3, 
                        height=800,
                        width=800,
                        title = "")

fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [35]:
import plotly.express as px
#to see correlation between number of violations in census block vs number of fecs in census blocks
# if one overpowers, it is difficult to see that. So we are scaling up fec address counts.


fig = px.scatter_mapbox(plot_df, 
                        lat='lat_avg', 
                        lon='long_avg', 
                        color='category',
                        #color_discrete_sequence=px.colors.qualitative.Pastel2,
                        color_discrete_sequence=['purple', 'green'],
                        size = 'num_addrs',
                        zoom=11.3, 
                        height=800,
                        width=1000,
                        title = "")

fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [20]:
census_blocks['Tract Code'].unique()

array([6.40100e+03, 2.03060e+04, 2.01030e+04, 4.30200e+03, 4.00500e+05,
       4.80100e+03, 5.30300e+03, 3.60200e+03, 7.00000e+03, 2.30000e+03,
       3.60100e+03, 3.30100e+03, 5.70000e+03, 2.10010e+04, 1.00000e+03,
       2.60000e+03, 3.10300e+03, 6.80000e+03, 1.70300e+03, 2.10100e+03,
       6.70000e+03, 4.00000e+03, 6.60000e+03, 6.50000e+03, 3.40000e+03,
       8.00000e+02, 4.70000e+03, 5.20100e+03, 2.00000e+03, 3.90200e+03,
       4.50000e+03, 2.02020e+04, 5.50100e+03, 1.00000e+02, 6.30100e+03,
       9.70300e+05, 5.40100e+03, 1.02030e+04, 2.90200e+03, 5.20200e+03,
       3.70000e+03, 2.70000e+03, 2.80000e+03, 7.00000e+02, 2.90100e+03,
       3.80100e+03, 6.30200e+03, 5.30400e+03, 2.20000e+03, 5.30600e+03,
       3.14020e+04, 4.40300e+03, 5.90000e+03, 5.10000e+03, 3.90100e+03,
       9.80000e+05, 4.00000e+02, 2.01020e+04, 3.20000e+03, 2.01010e+04,
       5.00000e+02, 1.02040e+04, 3.30200e+03, 6.00000e+02, 6.40200e+03,
       3.80200e+03, 2.03030e+04, 2.01040e+04, 5.60000e+03, 2.020